In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pca_ecog import PCA_Ecog_preprocessed
import warnings

ImportError: No module named pca_ecog

In [ ]:
class Regression_Pipeline():
    def __init__(pat_name,sess):
        #create
        edf_path, emo_path=self.get_paths(pat_name,sess)
        self.pecog=PCA_Ecog_preprocessed(edf_path)
        #TODO this part
        self.emo_data=pd.read_hdf(emo_path)
        
    def get_paths(self, pat_name,sess):
        #path for preprocessed edf data. THIS ASSUMES CODE IS RUN ON CYLON
        edf_path=os.path.join(os.path.sep,'/data2/users/stepeter/Preprocessing/Reref','preprocessed_'+pat_name+'_'+sess+'.h5')
        #path for emo annotations. THIS ASSUMES CODE IS WHERE THIS LINKS,
        emo_path=os.path.join(os.path.sep,'/home/emil/data/hdf_data',pat_name+'_'+sess+'.h5')
    
    def setup_range(start=0,stop=self.pecog.end)
        #for now, assume that both start at the same time. fix this as soon as real time is available
        
    